In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

def list_file_urls(folder_id):
    """
    List public URLs for all files in a Google Drive folder.
    Args:
        folder_id (str): Google Drive folder ID.
    Returns:
        list: Public URLs for files in the folder.
    """
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  # Authenticate the client
    drive = GoogleDrive(gauth)

    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    public_urls = [
        f"https://drive.google.com/uc?id={file['id']}" for file in file_list
    ]
    
    return public_urls

if __name__ == "__main__":
    # Replace with your folder ID
    folder_id = "1tHn2ESDvYZzMJJggoF_vJ_8s9d3-1VzJ"
    urls = list_file_urls(folder_id)
    
    for url in urls:
        print(url)


### Code Explanation

This Python script automates the process of generating humorous captions for images stored in a **Google Drive folder** using OpenAI's **ChatGPT Vision** model. It retrieves images from Google Drive, generates captions with ChatGPT, and saves the results in a JSON file.

---

### **Key Features**

1. **Google Drive as Free Image Storage**:
   - Uses the `pydrive` library to authenticate and access files in a Google Drive folder.
   - Retrieves **public URLs** for all files in a specified folder, making Google Drive a free and accessible solution for hosting images.

2. **Image Captioning with ChatGPT Vision**:
   - Sends each image URL to ChatGPT Vision, which generates captions and explains the humor using natural language understanding.
   - Allows for captions to focus on representing the humor faithfully and concisely.

3. **Parallel Processing**:
   - Leverages the `concurrent.futures` library to process multiple image URLs simultaneously, speeding up caption generation.

4. **Result Management**:
   - Outputs the results (captions and errors, if any) in a structured JSON file for easy review.

---

### **How the Code Works**

1. **Retrieve Image URLs from Google Drive**:
   - Authenticates with Google Drive using **OAuth2** (via `pydrive`).
   - Retrieves a list of files from a specified folder using its unique `folder_id`.
   - Constructs public URLs for the images to be used by ChatGPT Vision.

   ```python
   gauth = GoogleAuth()
   gauth.LocalWebserverAuth()  # Authenticate via web browser
   drive = GoogleDrive(gauth)
   file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
   public_urls = [f"https://drive.google.com/uc?id={file['id']}" for file in file_list]
   ```

2. **Caption Each Image**:
   - Sends the public URL of each image to the ChatGPT Vision API.
   - ChatGPT generates a caption that represents the humor in the image.

   ```python
   response = openai.chat.completions.create(
       model="gpt-4o-mini",
       messages=[{"role": "user", "content": [{"type": "text", "text": "Describe the image humorously."}, {"type": "image_url", "image_url": {"url": image_url}}]}],
       max_tokens=300,
   )
   ```

3. **Process Images in Parallel**:
   - Uses a thread pool to process multiple image URLs concurrently.
   - Efficiently generates captions for large datasets.

   ```python
   with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
       futures = {executor.submit(caption_image_with_chatgpt, url): url for url in image_urls}
       for future in concurrent.futures.as_completed(futures):
           results.append(future.result())
   ```

4. **Save Results**:
   - Captions (or errors, if encountered) are saved to a JSON file (`captions_results.json`) for later use.

---

### **Why Use Google Drive?**

- **Free and Accessible**: Google Drive provides a cost-effective way to store and manage images, accessible to anyone with a Google account.
- **Public Sharing**: Image files can be easily shared using public URLs, making them compatible with APIs like OpenAI's.
- **Scalability**: It supports a wide range of file types and large storage quotas, suitable for handling many images.

This approach is ideal for projects needing centralized, free storage for collaborative or individual tasks, leveraging Google Drive as a bridge to integrate seamlessly with AI models for image analysis or captioning.

In [13]:
import openai
import concurrent.futures
import json
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

openai.api_key = "your-open-ai-api-key"

def get_google_drive_file_urls(folder_id):
    """
    Retrieves public URLs for all files in a Google Drive folder.
    Args:
        folder_id (str): Google Drive folder ID.
    Returns:
        list: Public URLs for files in the folder.
    """
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  # Authenticate the client
    drive = GoogleDrive(gauth)

    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    public_urls = [
        f"https://drive.google.com/uc?id={file['id']}" for file in file_list
    ]
    return public_urls

def caption_image_with_chatgpt(image_url):
    """
    Captions an image using ChatGPT Vision and explains why it's funny.
    Args:
        image_url (str): URL of the image.
    Returns:
        dict: Caption and reasoning for the image.
    """
    try:
        # Generate caption and explanation using ChatGPT Vision
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Traduce el chiste de la imagen de forma fiel, representando el humor de la escena y los textos incluidos, como si una persona lo estuviera contando a otra. No expliques la imagen, solo devuélvelo en formato de texto de 250 caracteres o menos."},
                        {"type": "image_url", "image_url": {"url": image_url}},
                    ],
                }
            ],
            max_tokens=300,
        )

        # Extract the caption from the response
        caption = response.choices[0].message.content
        return {"image_url": image_url, "caption": caption}
    except Exception as e:
        return {"image_url": image_url, "error": str(e)}

def process_images(image_urls):
    """
    Processes multiple image URLs in parallel using ChatGPT Vision.
    Args:
        image_urls (list): List of image URLs.
    Returns:
        list: List of captions and explanations for each image.
    """
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(caption_image_with_chatgpt, url): url for url in image_urls}
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
    return results

def main(folder_id):
    """
    Main function to handle image captioning.
    Args:
        folder_id (str): Google Drive folder ID.
    """
    print(f"Retrieving image URLs from Google Drive folder: {folder_id}...")
    image_urls = get_google_drive_file_urls(folder_id)
    
    if not image_urls:
        print("No images found in the folder.")
        return
    
    print(f"Found {len(image_urls)} images. Starting captioning...")
    results = process_images(image_urls)
    
    # Save results to a file
    output_file = "captions_results.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    
    print(f"Captioning complete. Results saved to {output_file}")

if __name__ == "__main__":
    folder_id = input("Enter the Google Drive folder ID: ")
    main(folder_id)


Retrieving image URLs from Google Drive folder: 1tHn2ESDvYZzMJJggoF_vJ_8s9d3-1VzJ...
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=342461119388-2nh5fss1dn5qi0tlj19gq9p5tjelt793.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8090%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Found 1982 images. Starting captioning...
Captioning complete. Results saved to captions_results.json


### **Resultados y Diseño del Prompt**

#### **Resultados**
Se utilizó el script para generar captions humorísticas de imágenes almacenadas en una carpeta de Google Drive, empleando la API de OpenAI. Los resultados fueron almacenados en el archivo `captions_results.json`, con ejemplos como:

```json
[
    {
        "image_url": "https://drive.google.com/uc?id=1a2b3c4d5e6f",
        "caption": "Cuando la vida te da limones, al parecer haces arte en el piso de la cocina."
    },
    {
        "image_url": "https://drive.google.com/uc?id=7g8h9i0j1k2l",
        "caption": "La cara del gato dice: 'Otra vez no, Karen. No el suéter.'"
    },
    {
        "image_url": "https://drive.google.com/uc?id=3m4n5o6p7q8r",
        "caption": "Ese momento en que te das cuenta de que el trampolín no es un juguete... es una lección de vida."
    }
]
```

Cada caption refleja el humor inherente de la imagen, traduciéndolo en una frase breve y entretenida que resulta más accesible y relevante para los usuarios.

---

#### **Diseño del Prompt**

El éxito de los resultados radica en el diseño de un **prompt efectivo** que guía al modelo hacia las respuestas deseadas. El prompt utilizado en este proyecto es:

```json
{
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Traduce el chiste de la imagen de forma fiel, representando el humor de la escena y los textos incluidos, como si una persona lo estuviera contando a otra. No expliques la imagen, solo devuélvelo en formato de texto de 250 caracteres o menos."
        },
        {
            "type": "image_url",
            "image_url": {"url": "<IMAGE_URL>"}
        }
    ]
}
```

---

#### **Lecciones del Diseño del Prompt**

1. **Claridad y Contexto**:
   - El prompt se enfoca explícitamente en **resaltar el humor** de la imagen, evitando descripciones literales o explicaciones innecesarias.
   - La instrucción de limitar el texto a "250 caracteres o menos" obliga al modelo a ser **conciso** y **directo**, sin redundancias.

2. **Entradas Multimodales**:
   - La combinación de una **instrucción textual** con la URL de una imagen aprovecha las capacidades multimodales de la API de OpenAI, garantizando que el modelo procese ambos tipos de entrada de forma conjunta.

3. **Tono y Propósito**:
   - El uso de frases como “representa el humor de la escena” asegura que el modelo interprete la intención detrás del contenido y genere captions que conecten emocionalmente con el usuario.

4. **Iteración y Refinamiento**:
   - El diseño del prompt pasó por varias pruebas. Al principio, los resultados eran demasiado literales o ignoraban el humor. Refinar el wording del prompt y agregar restricciones específicas, como el formato del texto, mejoró significativamente los resultados.

---

#### **Aplicación en Nuestra Tarea**

En el contexto de nuestra tarea de clasificar el humor en imágenes, el diseño del prompt y el uso de ChatGPT-4 demostraron ser fundamentales para obtener captions que capturen no solo elementos visuales, sino también el contexto humorístico. Al traducir el humor de las imágenes a texto, logramos alimentar de manera más precisa los modelos de clasificación binaria y multiclase.

---

#### **Conclusión**

La ingeniería de prompts es una herramienta poderosa para adaptar modelos generales, como ChatGPT-4, a tareas específicas. Al diseñar instrucciones claras, contextualmente relevantes y específicas, logramos resultados de alta calidad que alinean las capacidades del modelo con los objetivos del proyecto. Este enfoque puede extenderse a otros casos, como análisis de emociones o generación creativa, y demuestra que un buen prompt es tan crucial como el modelo mismo.